# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [7]:
# git_token = "YOUR_GIT_TOKEN"
# short_code = "YOUR_SHORT_CODE"

# # Check the current python version (It should be using Python 3.10) and update pip to the latest version.
# !python --version
# !python -m pip install --user --upgrade pip

# # Clone MASE from your branch (the branch must already exist)
# !git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# # Install requirements
# !python -m pip install -r ./mase/machop/requirements.txt

# # Change working directory to machop
# %cd ./mase/machop/

In [8]:
# !./ch --help

# General introduction

In this lab, you will learn how to use the search functionality in the software stack of MASE.

There are in total 4 tasks you would need to finish.

# Writing a search using MaseGraph Transforms

In this section, our objective is to gain a comprehensive understanding of the construction of the current search function in Mase. To achieve this, we will require these essential components:

- MaseGraph: This component should be already created in the preceding lab.
- Search space: This component encompasses and defines the various available search options.
- Search strategy: An implementation of a search algorithm.
- Runner: This vital component manages and executes training, evaluation, or both procedures while generating a quality metric.

By analyzing these components, we can delve into the workings and effectiveness of the existing search function in Mase.

#Turning your network to a graph

We follow a similar procedure of what you have tried in lab2 to now produce a MaseGraph, this is converted from your pre-trained JSC model:

In [9]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
# machop_path = Path(".").resolve().parent.parent /"machop"
# assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
# sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model




set_logging_verbosity("info")

batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

INFO     Set logging level to info


#Defining a search space

Based on the previous `pass_args` template, the following code is utilized to generate a search space. The search space is constructed by combining different weight and data configurations in precision setups.

In [10]:
pass_args = {
"by": "type",
"default": {"config": {"name": None}},
"linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
},}

import copy
# build a search space
data_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
w_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
search_spaces = []
for d_config in data_in_frac_widths:
    for w_config in w_in_frac_widths:
        pass_args['linear']['config']['data_in_width'] = d_config[0]
        pass_args['linear']['config']['data_in_frac_width'] = d_config[1]
        pass_args['linear']['config']['weight_width'] = w_config[0]
        pass_args['linear']['config']['weight_frac_width'] = w_config[1]
        # dict.copy() and dict(dict) only perform shallow copies
        # in fact, only primitive data types in python are doing implicit copy when a = b happens
        search_spaces.append(copy.deepcopy(pass_args))

## Defining a search strategy and a runner

The code provided below consists of two main `for` loops. The first `for` loop executes a straightforward brute-force search, enabling the iteration through the previously defined search space.

In contrast, the second `for` loop retrieves training samples from the train data loader. These samples are then utilized to generate accuracy and loss values, which serve as potential quality metrics for evaluating the system's performance.


In [11]:
#Assuming the initialization and import statements remain the same
import torch
import os
import time
from torchmetrics.classification import MulticlassAccuracy
from thop import profile  # Make sure to install thop

# Rewrite of measure functions to not use device explicitly
def measure_latency(model, dataloader, iterations=10):
    model.eval()
    device = next(model.parameters()).device  # Infer device from model
    total_time = 0
    
    with torch.no_grad():
        for i, (inputs, _) in enumerate(dataloader):
            if i >= iterations:
                break
            inputs = inputs.to(device)
            start_time = time.time()
            _ = model(inputs)
            total_time += time.time() - start_time

    avg_latency = total_time / iterations
    return avg_latency

def measure_model_size(model, path="model.pth"):
    torch.save(model.state_dict(), path)
    model_size = os.path.getsize(path)  # Size in bytes
    return model_size

def measure_flops(model, inputs):
    model.eval()
    device = next(model.parameters()).device  # Infer device from model
    with torch.no_grad():
        # Ensure inputs is a single tensor or a tuple of tensors
        if isinstance(inputs, list):
            inputs = torch.stack(inputs).to(device)  # Ensure inputs are moved to the same device
        else:
            inputs = inputs.to(device)
        if not isinstance(inputs, tuple):
            inputs = (inputs,)
        flops, _ = profile(model, inputs=inputs)
    return flops

def measure_memory_usage(model, inputs):
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
        model.to('cuda')  # Ensure model is on GPU
        inputs = inputs.to('cuda')  # Ensure inputs are on GPU
        with torch.no_grad():
            torch.cuda.synchronize()  # Wait for all kernels to finish
            before = torch.cuda.memory_allocated()
            _ = model(inputs)
            torch.cuda.synchronize()  # Wait for all kernels to finish
            after = torch.cuda.memory_allocated()
        memory_usage = after - before
    else:
        print("CUDA is not available. Memory usage can only be measured on GPU.")
        memory_usage = None
    return memory_usage

In [12]:
# grid search
import torch
from torchmetrics.classification import MulticlassAccuracy

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

metric = MulticlassAccuracy(num_classes=5)
num_batches = 5
recorded_metrics = []
recorded_accs = []
for i, config in enumerate(search_spaces):
    mg, _ = quantize_transform_pass(mg, config)
    j = 0
    acc_avg, loss_avg = 0, 0
    accs, losses = [], []
    for inputs in data_module.train_dataloader():
        xs, ys = inputs
        device = xs.device  # Infer device from input tensors
        preds = mg.model(xs)
        loss = torch.nn.functional.cross_entropy(preds, ys)
        acc = metric(preds, ys)
        accs.append(acc)
        losses.append(loss)
        if j > num_batches:
            break
        j += 1
    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    recorded_accs.append(acc_avg)

    # No explicit device usage for measurement functions
    latency = measure_latency(mg.model, data_module.train_dataloader())
    model_size = measure_model_size(mg.model)
    inputs, _ = next(iter(data_module.train_dataloader()))
    flops = measure_flops(mg.model, inputs)
    print(inputs)


     # Measure memory usage
    #inputs, _ = next(iter(data_module.train_dataloader()))  # Get a batch of data
    device = next(mg.model.parameters()).device
    input_=inputs.to('cuda')
    memory_usage = measure_memory_usage(mg.model,input_ )


    # Combine and record metrics
    combined_metric = acc_avg  # Placeholder for actual combination logic
    recorded_metrics.append({
    'accuracy': acc_avg,
    'loss': loss_avg,
    'latency': latency,
    'model_size': model_size,
    'flops': flops,
    'memory_usage': memory_usage,  # Add memory usage to the recorded metrics
    'combined_metric': combined_metric
    })

[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
tensor([[ 0.4646,  0.1247, -0.5255, -0.6368, -0.3731, -0.4979,  0.2682, -0.3610,
          0.2682, -0.0336, -0.0775, -0.5593,  0.2305, -0.3172, -0.5843, -0.8917],
        [ 1.4115, -1.4242, -0.2565, -0.1675,  0.2041, -0.2514,  1.0959, -0.4740,
          1.0959, -1.2285, -1.4656, -1.2409, -0.7380, -1.0417,  0.0902, -0.6505],
        [ 1.3058, -0.6052, -0.9499, -0.7939, -0.5723, -0.5199,  2.2835,  2.0472,
          2.2835,  0.4518, -0.2803, -0.3654,  0.8312,  0.5013, -0.7243, -0.9399],
        [-0.0433,  0.2513,  0.3254, -0.0555, -0.0467, -0.3526, -0.7660, -0.8514,
         -0.7660, -0.4496, -0.3113, -0.6449, -0.4554, -0.7209,  0.2827, -0.1682],
        [ 0.1697,  0.4937, -0.6021, -0.6969, -0.7017, -0.5662, -0.7677, -0.7360,
         -0.7677, -0.5224, -0.4983, -0.6678, -0.6409, -0.6992, -0.6392, -0.8917],
        [-0.3469, 

NameError: name 'inputs_' is not defined

We now have the following task for you:

1. Explore additional metrics that can serve as quality metrics for the search process. For example, you can consider metrics such as latency, model size, or the number of FLOPs (floating-point operations) involved in the model.

2. Implement some of these additional metrics and attempt to combine them with the accuracy or loss quality metric. It's important to note that in this particular case, accuracy and loss actually serve as the same quality metric (do you know why?).



# The search command in the MASE flow

The search flow implemented in MASE is very similar to the one that you have constructed manually, the overall flow is implemented in [search.py](../../machop/chop/actions/search/search.py), the following bullet points provide you pointers to the code base.

- MaseGraph: this is the [MaseGraph](../../machop/chop/passes/graph/mase_graph.py) that you have used in lab2.
- Search space: The base class is implemented in [base.py](../../machop/chop/actions/search/search_space/base.py) , where in the same folder you can see a range of different supported search spaces.
- Search strategy: Similar to the search space, you can find a a base class [definition](../../machop/chop/actions/search/strategies/base.py), where different strategies are also defined in the same folder.
- Runner: Different [runners](../../machop/chop/actions/search/strategies/runners) can produce different metrics, they may also use `transforms` to help compute certain search metrics.

This enables one to execute the search through the MASE command line interface, remember to change the name after the `--load` option.


In [ ]:
!./ch search --config configs/examples/jsc_toy_by_type.toml --load your_pre_trained_ckpt

/bin/bash: ./ch: No such file or directory


In this scenario, the search functionality is specified in the `toml` configuration file rather than via command-line inputs. This approach is adopted due to the multitude of configuration parameters that need to be set; encapsulating them within a single, elegant configuration file enhances reproducibility.

In `jsc_toy_by_type.toml`, the `search_space` configuration is set in `search.search_space`, the search strategy is configured via `search.strategy`. If you are not familiar with the `toml` syntax, you can read [here](https://toml.io/en/v1.0.0).

> In order to accomplish the following task, it is necessary to make direct modifications to the code base. This can be challenging within the Colab environment. **It is recommended to implement the task on a local setup and utilize Colab strictly as a server to execute the search command above.** Consider Colab as a dedicated server for this purpose.

With now an understanding of how the MASE flow work, consider the following tasks

3. Implement the brute-force search as an additional search method within the system, this would be a new search strategy in MASE.
4. Compare the brute-force search with the TPE based search, in terms of sample efficiency. Comment on the performance difference between the two search methods.